In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
import cv2

In [2]:
DATA_DIR="E:\Python\Spoofing-Version02\images"
CATEGORIES=["fake","real"]

In [3]:
IMG_SIZE=160

In [4]:
training_data=[]
def create_training_data():
    for category in CATEGORIES:
        path=os.path.join(DATA_DIR,category) #path to cats and dogs
        class_num=CATEGORIES.index(category)
        for img in os.listdir(path):
            try:
                img_arr=cv2.imread(os.path.join(path,img))
                new_array=cv2.resize(img_arr,(IMG_SIZE,IMG_SIZE))
                training_data.append([new_array,class_num])
            except Exception as e:
                pass
    
create_training_data()

In [5]:
print(len(training_data))

6389


In [6]:
import random

random.shuffle(training_data)

In [7]:
for sample in training_data[:5]:
    print(sample[1])

1
1
1
1
1


In [8]:
X=[]
y=[]

for features,label in training_data:
    X.append(features)
    y.append(label)
    
#how many features did we have -1 saying it could be many any number,we know shape of data imgsize x imgsize
#and than one bcz of grayscale image
#LAST 1 WILL BE 3 IF USING RGB
X=np.array(X).reshape(-1,IMG_SIZE,IMG_SIZE,3)
print(X.shape)

(6389, 160, 160, 3)


In [9]:
X=X/255.0

In [10]:
X_train=X[:4999]
y_train=y[:4999]
X_test=X[4999:]
y_test=y[4999:]
print(X_train.shape)

(4999, 160, 160, 3)


In [11]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense,Dropout,Activation,Flatten,Conv2D,MaxPooling2D,GlobalAveragePooling2D
import pickle
from tensorflow.keras import applications
from tensorflow.keras.models import Model
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.preprocessing import image


In [12]:
vgg_model = applications.VGG16(weights='imagenet',
                               include_top=False,
                               input_shape=(160 ,160, 3))



In [13]:
x=vgg_model.output
x=GlobalAveragePooling2D()(x)
x=Dense(512,activation='relu')(x) #we add dense layers so that the model can learn more complex functions and classify for better results.
x=Dense(512,activation='relu')(x) #dense layer 2
x=Dense(64,activation='relu')(x) #dense layer 3
preds=Dense(1,activation='sigmoid')(x) #final layer with softmax activation

In [14]:
model=Model(inputs=vgg_model.input,outputs=preds)

In [15]:
for i,layer in enumerate(model.layers):
    print(i,layer.name)

0 input_1
1 block1_conv1
2 block1_conv2
3 block1_pool
4 block2_conv1
5 block2_conv2
6 block2_pool
7 block3_conv1
8 block3_conv2
9 block3_conv3
10 block3_pool
11 block4_conv1
12 block4_conv2
13 block4_conv3
14 block4_pool
15 block5_conv1
16 block5_conv2
17 block5_conv3
18 block5_pool
19 global_average_pooling2d
20 dense
21 dense_1
22 dense_2
23 dense_3


In [16]:
for layer in model.layers[:20]:
    layer.trainable=False
for layer in model.layers[20:]:
    layer.trainable=True


In [17]:
from tensorflow.keras.optimizers import Adam

In [18]:
model.compile(optimizer='Adam',loss='binary_crossentropy',metrics=['accuracy'])

In [20]:
model.fit(X_train,np.array(y_train),batch_size=5,epochs=3,validation_split=0.2)

Train on 3999 samples, validate on 1000 samples
Epoch 1/3
3999/3999 [==============================] - 1746s 437ms/sample - loss: 0.0133 - accuracy: 0.9937 - val_loss: 8.6084e-07 - val_accuracy: 1.0000
Epoch 2/3
3999/3999 [==============================] - 1868s 467ms/sample - loss: 2.4169e-07 - accuracy: 1.0000 - val_loss: 1.2105e-07 - val_accuracy: 1.0000
Epoch 3/3
3999/3999 [==============================] - 1674s 419ms/sample - loss: 6.1882e-08 - accuracy: 1.0000 - val_loss: 2.7484e-08 - val_accuracy: 1.0000


In [21]:
import h5py
model.save('Vgg_model.h5')

In [22]:
pred = model.predict([X_test])

In [27]:
n=0
print(y_test[n])

1


In [31]:
#print(np.argmax(pred[n]))
print(pred[n])

[1.]


In [32]:
def func(pred):
    if pred==0:
        return "fake"
    else :
        return "real"
    

In [35]:
IMG_SIZE=160
def CNN_frame_shape(img):
    img=cv2.resize(img,(IMG_SIZE,IMG_SIZE))
    img_new = img.reshape((-1,img.shape[0],img.shape[1],3))
    return img_new


In [80]:
def func_CNN(pred):
    chk=pred[0][0]
    if chk < 0.01:
        result = "fake"
    else:
        result = "real"
    return result

In [109]:
#my_img=cv2.imread("E:\Python\liveness_detection\Hannan-13.jpeg",cv2.IMREAD_GRAYSCALE)
#my_img=cv2.imread("E:\Python\liveness_detection\mahadFriend-Fake.jpeg",cv2.IMREAD_GRAYSCALE)
#my_img=cv2.imread("E:\Python\Spoofing\Wasif-02.jpg",cv2.IMREAD_GRAYSCALE)
#my_img=cv2.imread("E:\Python\liveness_detection\justin.jpg",cv2.IMREAD_GRAYSCALE)
#my_img=cv2.imread(r"E:\Python\Spoofing-Version02\images\fake\Old Hannan576.jpg",cv2.IMREAD_GRAYSCALE)
my_img=cv2.imread("E:\Python\Spoofing-Version02\Fake-Hannan.jpeg",cv2.IMREAD_GRAYSCALE)
#my_img=cv2.imread(r"E:\Python\liveness_detection\images\fake\img1.jpg",cv2.IMREAD_GRAYSCALE)
#my_img=cv2.imread("E:\Python\liveness_detection\justin_fake.jpg",cv2.IMREAD_GRAYSCALE)
#my_img=cv2.imread("E:\Python\Spoofing-Version02\Maaz-Fake-3.jpg")

In [110]:
my_img=cv2.cvtColor(my_img,cv2.COLOR_BGR2RGB)

In [111]:
roi = cv2.resize(my_img, (160, 160))
#print(roi)
roi=roi/255

In [112]:
f_img=CNN_frame_shape(roi)
print(f_img.shape)
pred = model.predict(f_img)
print(pred)
var=func_CNN(pred)
print(var)

(1, 160, 160, 3)
[[0.99999475]]
real
